# Tutorial #1: Interfacing w/ the Library

**Target Audience:** Design Engineers / Front-end User

**Learning Objectives:**
1. Primary Functionality
    1. [Selecting a component](#component)
    2. [Meaning / notation of characteristics](#reading-characteristics)
    3. [Finding geometrical parameters which best suits target characteristics](#select-characteristics)
2. Secondary Functionality
    1. Seeing who has contributed what data

# Section 1: Primary Functionality

## Section 1.1: Selecting a Component <a id="component"></a>

First, let's see which components are currently supported.

In [1]:
import metal_library

print(metal_library.supported_components)

['TransmonCross', 'TransmonPocket']


Now we can place one of the supported components into `Reader`. This is a class meant to parse the library (located at `C:\\...\metal_library\library`)

In [2]:
from metal_library import Reader

reader = Reader(component_name='TransmonCross')
print(reader)

Interfacing with the library will be conducted through this object.

## Section 1.2: Understanding Structure of Characteristics <a id="reading-characteristics"></a>

Since we're conducting science, we must use exact language when talking about the characteristics we're pulling.

Example of **imprecise** language:
- 👎 "Get the coupling strength?"

Example of **precise** language:
- 👍 "Get the coupling strength between the readout cavity and qubit"
- 👍 "Get the coupling strength between qubit #1 and the bus (connecting qubit #1 and qubit #2)"

The following attributes / functions were created to help the front-end user follow precisely what each component characteristic means.



### Component Types
In simulations, we don't always simulate the entire chip at once. So there's a property `component_types` to view which combinations we've simulated.

It also includes a little blurb to help clarify what the names imply!

In [3]:
reader.component_types

╒════════════════════════════╤════════════════════════════════════════════════╕
│ Supported Component Type   │ Blurb                                          │
╞════════════════════════════╪════════════════════════════════════════════════╡
│ QubitOnly                  │ Qubit only                                     │
├────────────────────────────┼────────────────────────────────────────────────┤
│ QubitCavity                │ Qubit capacitvely coupled to a readout cavity. │
├────────────────────────────┼────────────────────────────────────────────────┤
│ QubitDriveline             │ Qubit capacitvely coupled to a drive line.     │
╘════════════════════════════╧════════════════════════════════════════════════╛


['QubitOnly', 'QubitCavity', 'QubitDriveline']

### Characteristics of Component Type

Within each `component_type`, the types of observables / component characteristics you can extract change. To see which characteristics are supported, we can use `.get_characteristic_info`.

In [4]:
reader.get_characteristic_info(component_type="QubitOnly");

╒═════════════════════════╤═══════════════════════════════════════╤═════════╤═══════════════╕
│ CSV Column Name         │ Description                           │ Units   │ Math Symbol   │
╞═════════════════════════╪═══════════════════════════════════════╪═════════╪═══════════════╡
│ Qubit_Frequency_GHz     │ Qubit 01 Transition Freq (Linear GHz) │ GHz     │ f_{q,01}      │
├─────────────────────────┼───────────────────────────────────────┼─────────┼───────────────┤
│ Qubit_Anharmonicity_MHz │ Qubit Anharmonicity (Linear MHz)      │ MHz     │ \alpha_{q}    │
╘═════════════════════════╧═══════════════════════════════════════╧═════════╧═══════════════╛


In [5]:
reader.get_characteristic_info(component_type="QubitCavity");

╒═════════════════════════╤════════════════════════════════════════════════╤═════════╤═══════════════╕
│ CSV Column Name         │ Description                                    │ Units   │ Math Symbol   │
╞═════════════════════════╪════════════════════════════════════════════════╪═════════╪═══════════════╡
│ Qubit_Frequency_GHz     │ Qubit 01 Transition Freq (Linear GHz)          │ GHz     │ f_{q,01}      │
├─────────────────────────┼────────────────────────────────────────────────┼─────────┼───────────────┤
│ Qubit_Anharmonicity_MHz │ Qubit Anharmonicity (Linear MHz)               │ MHz     │ \alpha_{q}    │
├─────────────────────────┼────────────────────────────────────────────────┼─────────┼───────────────┤
│ Cavity_Frequency_GHz    │ Cavity Resonant Freq (Linear MHz)              │ GHz     │ _{c}          │
├─────────────────────────┼────────────────────────────────────────────────┼─────────┼───────────────┤
│ Coupling_Strength_MHz   │ Coupling between qubit and cavity (Linear MHz

In [6]:
reader.get_characteristic_info(component_type="QubitDriveline");

╒══════════════════════════╤═════════════════════════════════════════════╤═════════╤═══════════════╕
│ CSV Column Name          │ Description                                 │ Units   │ Math Symbol   │
╞══════════════════════════╪═════════════════════════════════════════════╪═════════╪═══════════════╡
│ Qubit_Frequency_GHz      │ Qubit 01 Transition Freq (Linear GHz)       │ GHz     │ f_{q,01}      │
├──────────────────────────┼─────────────────────────────────────────────┼─────────┼───────────────┤
│ Qubit_Anharmonicity_MHz  │ Qubit Anharmonicity (Linear MHz)            │ MHz     │ \alpha_{q}    │
├──────────────────────────┼─────────────────────────────────────────────┼─────────┼───────────────┤
│ Driveline_Decay_Rate_MHz │ Feedline limited T1 Decay Rate (linear MHz) │ MHz     │ \kappa        │
╘══════════════════════════╧═════════════════════════════════════════════╧═════════╧═══════════════╛


## Section 1.3: Finding a Geometry for Given Characteristics (MAIN FUNCTIONALITY)<a id="select-characteristics"></a>

The main functionality of this program is to find a geometry for a set of target characteristics, let's now dive into it!

First you must select a component_type and read that library.

In [7]:
reader.read_library(component_type='QubitOnly')

In [8]:
reader.library.characteristic.head()

,Qubit_Frequency_GHz,Qubit_Anharmonicity_MHz,misc
0,4368.966835,208.731884,{'project_info': {'pinfo': design_name ...
1,4368.966835,208.731884,{'project_info': {'pinfo': design_name ...
2,4368.966835,208.731884,{'project_info': {'pinfo': design_name ...
3,4193.934789,210.539654,{'project_info': {'pinfo': design_name ...
4,4031.690829,211.554005,{'project_info': {'pinfo': design_name ...


### Now we can interface with the library using the `Selector` class.

In [9]:
from metal_library import Selector

selector = Selector(reader)
print(selector)

Let's select a geometry which results in the closest qubit characteristics.

Call `Selector.find_closest`

```
Main functionality. Select the closest geometry for a set of characteristics.

Args:
    target_params (dict): A dictionary where the keys are the column names in `self.characteristic`,
                          and the values are the target values to compare against.
    num_top (int): The number of rows with the smallest Euclidean distances to return.
    metric (str, optional): Metric to determine closeness. Defaults to "Euclidian". 
                            Must choose from `self.__supported_metrics__`.
    display (boo, optional): Print out results? Defaults to True.

Returns:
    best_geometries (list[dict]): Geometries in the style of QComponent.options. Ranked closest to furthest.
```


In [10]:
# Here are the supported metrics...
selector.__supported_metrics__

['Euclidian', 'Manhattan', 'Chebyshev']

In [11]:
best_geometries = selector.find_closest(target_params={"Qubit_Frequency_GHz": 4200,
                                                       "Qubit_Anharmonicity_MHz": 200},
                                        num_top=3,
                                        metric="Euclidian",
                                        display=True)

Here are the top 3 geometries using the Euclidian metric.
Target parameters: {'Qubit_Frequency_GHz': 4200, 'Qubit_Anharmonicity_MHz': 200} 

The best match was:
{'Unnamed: 0': 531, 'pos_x': 0, ' pos_y': '0.0um', ' orientation': 0, ' chip': 'main', ' layer': 1, ' connection_pads': {'readout': {'connector_type': 0, 'claw_length': '205um', 'ground_spacing': '10um', 'claw_width': '15um', 'claw_gap': '5.1um', 'claw_cpw_length': '40um', 'claw_cpw_width': '10um', 'connector_location': 180}}, ' cross_width': '30um', ' cross_length': '185um', ' cross_gap': '29um', ' hfss_inductance': '10nH', ' hfss_capacitance': 0, ' hfss_resistance': 0, ' hfss_mesh_kw_jj': 7e-06, ' q3d_inductance': '10nH', ' q3d_capacitance': 0, ' q3d_resistance': 0, ' q3d_mesh_kw_jj': 7e-06, ' gds_cell_name': 'my_other_junction', ' aedt_q3d_inductance': 1e-08, ' aedt_q3d_capacitance': 0, ' aedt_hfss_inductance': 1.3e-08, ' aedt_hfss_capacitance': 0} 

The other matches (ordered closeset to furthest):
Match #2:
{'Unnamed: 0': 